In [1]:
import os
import re
import torch
import random
import numpy as np
import pandas as pd

from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
from konlpy.tag import Mecab, Okt

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# 1.데이터 처리 과정 살펴보기

In [3]:
data_dir = './data/ChatbotData.csv'
df = pd.read_csv(data_dir) ## Question, Answer, Label(emotion - 0 : normal, 1 : negative, 2 : positive)
print(df)

                             Q                         A  label
0                       12시 땡!                하루가 또 가네요.      0
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0
4                      PPL 심하네                눈살이 찌푸려지죠.      0
...                        ...                       ...    ...
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2
11820              흑기사 해주는 짝남.                    설렜겠어요.      2
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2

[11823 rows x 3 columns]


In [4]:
question = df['Q']
answer = df['A']

print(len(question), len(answer))

11823 11823


In [5]:
## 이 문자들을 제외한 나머지들은 모두 제거한다.
korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'
normalizer = re.compile(korean_pattern)
print(normalizer)

re.compile('[^ ?,.!A-Za-z0-9가-힣+]')


In [6]:
def normalize(sentence):
    return normalizer.sub("", sentence)

In [7]:
## 한글 형태소 분석
mecab = Mecab()
okt = Okt()

rand_idx = random.randint(0, len(question) - 1)
print(f"{question[rand_idx]} ---> {mecab.morphs(normalize(question[rand_idx]))}")

대리님이 너무 갈궈 ---> ['대리', '님', '이', '너무', '갈궈']


In [8]:
def clean_text(sentence, tagger):
    sentence = normalize(sentence)
    sentence = tagger.morphs(sentence)
    sentence = ' '.join(sentence)
    sentence = sentence.lower()

    return sentence

print(clean_text(question[rand_idx], okt))
print(clean_text(answer[rand_idx], okt))

대리 님 이 너무 갈궈
더 웃으면서 대해 보세요 .


In [9]:
questions = [clean_text(sent, okt) for sent in question[:1000]]
answers = [clean_text(sent, okt) for sent in answer[:1000]]

print(questions[:5])
print(answers[:5])

['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', 'ppl 심하네']
['하루 가 또 가네요 .', '위로 해 드립니다 .', '여행 은 언제나 좋죠 .', '여행 은 언제나 좋죠 .', '눈살 이 찌푸려지죠 .']


In [10]:
## 단어 사전 생성
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2

class WordVocab():
    def __init__(self):
        self.word2count = {}
        
        self.word2index = {'<PAD>' : PAD_TOKEN,
                           '<SOS>' : SOS_TOKEN,
                           '<EOS>' : EOS_TOKEN}
        
        self.index2word = {PAD_TOKEN : '<PAD>',
                           SOS_TOKEN : '<SOS>',
                           EOS_TOKEN : '<EOS'}
        self.n_words = 3

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [11]:
rand_idx = random.randint(0, len(questions) - 1)
print(f"Original : {questions[rand_idx]}")

lang = WordVocab()
lang.add_sentence(questions[rand_idx])
print(lang.word2index)

Original : 내 능력 이 너무 모자라
{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '내': 3, '능력': 4, '이': 5, '너무': 6, '모자라': 7}


In [12]:
## batch를 위해 문장의 길이를 맞춰준다. padding
max_length = 10
sentence_length = 6

sentence_tokens = np.random.randint(low=3, high=100, size=(sentence_length,))
sentence_tokens = sentence_tokens.tolist()
print(sentence_tokens)

[31, 25, 81, 47, 74, 63]


In [13]:
sentence_tokens = sentence_tokens[ : (max_length - 1)]
token_length = len(sentence_tokens)

print(sentence_tokens)
print(token_length)

[31, 25, 81, 47, 74, 63]
6


In [14]:
sentence_tokens.append(2) ## <EOS> token 추가.
for i in range(token_length, max_length - 1):
    sentence_tokens.append(PAD_TOKEN)

print(f"output : {sentence_tokens}")
print(f"total length: {len(sentence_tokens)}")

output : [31, 25, 81, 47, 74, 63, 2, 0, 0, 0]
total length: 10


# 2.Train, Test Dataset

In [15]:
class TextDataset(Dataset):
    def __init__(self, csv_path, min_length=3, max_length=32):
        super(TextDataset, self).__init__()

        ## Token 정의.
        self.PAD_TOKEN = 0
        self.SOS_TOKEN = 1
        self.EOS_TOKEN = 2

        self.tagger = Mecab() ## 형태소 분석기
        self.max_length = max_length ## 한 문장의 최대 길이

        df = pd.read_csv(csv_path)
        korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'
        self.normalizer = re.compile(korean_pattern)

        self.source_clean = []
        self.target_clean = []
        self.wordvocab = WordVocab()
        for _, row in df.iterrows():
            source = row['Q'] ## raw questions
            target = row['A'] ## raw answers

            source = self.clean_text(source)
            target = self.clean_text(target)

            if len(source.split()) > min_length and len(target.split()) > min_length:
                self.wordvocab.add_sentence(source)
                self.wordvocab.add_sentence(target)
                self.source_clean.append(source)
                self.target_clean.append(target)


    def __getitem__(self, idx):
        inputs = self.source_clean[idx] ## n번째 (형태소로 분리된) 문장 하나를 가져온다. ex) 내 의견 을 존중 해줬으면
        inputs_sentences = self.texts_to_sequences(inputs) ## ex) {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '내': 3, '의견': 4, '을': 5, '존중': 6, '해줬으면': 7}
        inputs_padded = self.pad_sequence(inputs_sentences)

        outputs = self.target_clean[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)

        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)
    
    
    def __len__(self):
        return len(self.source_clean)


    def normalize(self, sentence):
        return self.normalizer.sub("", sentence)


    def clean_text(self, sentence):
        sentence = self.normalize(sentence)
        sentence = self.tagger.morphs(sentence) ## 문장을 단어 단위로 분리한다.(형태소 분석)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()

        return sentence


    def texts_to_sequences(self, sentence):
        return [self.wordvocab.word2index[w] for w in sentence.split()] ## 각 형태소를 key로, index를 부여.


    def pad_sequence(self, sentence_tokens):
        sentence_tokens = sentence_tokens[ : (self.max_length - 1)]
        token_length = len(sentence_tokens)
        
        sentence_tokens.append(self.EOS_TOKEN)
        for i in range(token_length, (self.max_length - 1)):
            sentence_tokens.append(self.PAD_TOKEN)

        return sentence_tokens    

In [16]:
MAX_LENGTH = 50
dataset = TextDataset("./data/ChatbotData.csv", min_length=3, max_length=MAX_LENGTH)
x, y = dataset[10]
print(f"Question Tensor : {x.shape} \n {x}")
print(f"Answer Tensor : {y.shape} \n {y}")

Question Tensor : torch.Size([50]) 
 tensor([83, 84, 51, 85, 86, 18,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
Answer Tensor : torch.Size([50]) 
 tensor([87, 88, 58, 89, 63, 90, 11,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [17]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
print(train_size, test_size)

8168 2042


In [18]:
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset)

x, y = next(iter(train_dataloader))
print(x.shape, y.shape)

torch.Size([16, 50]) torch.Size([16, 50])


# 3.Seq2Seq model

## 3-1.embedding layer

In [19]:
embedding_dim = 64
embedding = nn.Embedding(num_embeddings=dataset.wordvocab.n_words, embedding_dim=embedding_dim) ## embedding할 단어의 수, embedding 차원

embedded = embedding(x)
print(f"{x.shape} --> {embedded.shape}")

# embedded = embedded.permute(1, 0, 2)
# print(embedded.shape)

torch.Size([16, 50]) --> torch.Size([16, 50, 64])


## 3-2.LSTM

In [20]:
hidden_dim = 32

lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, batch_first=True)
output, states = lstm(embedded)
print(output.shape, states[0].shape, states[1].shape)

torch.Size([16, 50, 32]) torch.Size([1, 16, 32]) torch.Size([1, 16, 32])


In [21]:
class Encoder(nn.Module):
    def __init__(self, num_vocabs, hidden_dim, embedding_dim, num_layers):
        super(Encoder, self).__init__()
        self.num_vocabs = num_vocabs
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers)

    def forward(self, x):
        x = self.embedding(x).permute(1, 0, 2)
        output, hidden = self.lstm(x)

        return output, hidden

encoder = Encoder(dataset.wordvocab.n_words, hidden_dim=hidden_dim, embedding_dim=embedding_dim, num_layers=1)
output, states = encoder(x)

print(output.shape, states[0].shape, states[1].shape)

torch.Size([50, 16, 32]) torch.Size([1, 16, 32]) torch.Size([1, 16, 32])


In [22]:
x = torch.abs(torch.randn(size=(1, 16)).long())
print(x.shape, x)

embedding_dim = 64
embedding = nn.Embedding(dataset.wordvocab.n_words, embedding_dim)

embedded = embedding(x)
print(embedded.shape)

lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, batch_first=True)
output, states = lstm(embedded)
print(output.shape, states[0].shape, states[1].shape)

output_layer = nn.Linear(32, dataset.wordvocab.n_words)
pred = output_layer(output[0])
print(output[0].shape)
print(pred.shape)

torch.Size([1, 16]) tensor([[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]])
torch.Size([1, 16, 64])
torch.Size([1, 16, 32]) torch.Size([1, 1, 32]) torch.Size([1, 1, 32])
torch.Size([16, 32])
torch.Size([16, 6417])


In [23]:
class Decoder(nn.Module):
    def __init__(self, num_vocabs, hidden_dim, embedding_dim, num_layers=1, dropout=0.2):
        super(Decoder, self).__init__()
        self.num_vocabs = num_vocabs
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers)
        self.out = nn.Linear(hidden_dim, num_vocabs)

    def forward(self, x, hidden_state):
        x = x.unsqueeze(0) ## (1, batch_size)
        embedded = F.relu(self.embedding(x))
        embedded = self.dropout(embedded)

        output, hidden = self.lstm(embedded, hidden_state)
        output = self.out(output.squeeze(0))

        return output, hidden
    
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words, hidden_dim=hidden_dim, embedding_dim=embedding_dim, num_layers=1)
x, y = next(iter(train_dataloader))

output, states = encoder(x)
print(output.shape, states[0].shape, states[1].shape)

x = torch.abs(torch.full(size=(16, ), fill_value=SOS_TOKEN).long())
print(x)

decoder_output, decoder_hidden = decoder(x, states)
print(decoder_output.shape, decoder_hidden[0].shape, decoder_hidden[1].shape)

torch.Size([50, 16, 32]) torch.Size([1, 16, 32]) torch.Size([1, 16, 32])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
torch.Size([16, 6417]) torch.Size([1, 16, 32]) torch.Size([1, 16, 32])


In [24]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        batch_size, output_length = outputs.shape
        output_num_vocabs = self.decoder.num_vocabs

        predicted_outputs = torch.zeros(output_length, batch_size, output_num_vocabs).to(self.device)
        _, decoder_hidden = self.encoder(inputs)
        decoder_input = torch.full((batch_size, ), SOS_TOKEN, device=self.device)

        for t in range(0, output_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            predicted_outputs[t] = decoder_output
            
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = decoder_output.argmax(1)
            decoder_input = outputs[:, t] if teacher_force else top1

        return predicted_outputs.permute(1, 0, 2)

In [25]:
encoder = Encoder(num_vocabs=dataset.wordvocab.n_words,
                  hidden_dim=32,
                  embedding_dim=64,
                  num_layers=1)

decoder = Decoder(num_vocabs=dataset.wordvocab.n_words,
                  hidden_dim=32,
                  embedding_dim=64,
                  num_layers=1)

seq2seq = Seq2Seq(encoder, decoder, device).to(device)
print(seq2seq)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6417, 64)
    (lstm): LSTM(64, 32)
  )
  (decoder): Decoder(
    (embedding): Embedding(6417, 64)
    (dropout): Dropout(p=0.2, inplace=False)
    (lstm): LSTM(64, 32)
    (out): Linear(in_features=32, out_features=6417, bias=True)
  )
)


In [26]:
x, y = next(iter(train_dataloader))
print(x.shape, y.shape)

output = seq2seq(x.to(device), y.to(device))
print(output.shape)

torch.Size([16, 50]) torch.Size([16, 50])
torch.Size([16, 50, 6417])


# 4.Training

In [27]:
NUM_VOCABS = dataset.wordvocab.n_words
HIDDEN_DIM = 512
EMBEDDING_DIM = 256

encoder = Encoder(num_vocabs=NUM_VOCABS,
                  hidden_dim=HIDDEN_DIM,
                  embedding_dim=EMBEDDING_DIM,
                  num_layers=1)

decoder = Decoder(num_vocabs=NUM_VOCABS,
                  hidden_dim=HIDDEN_DIM,
                  embedding_dim=EMBEDDING_DIM,
                  num_layers=1)

model = Seq2Seq(encoder.to(device), decoder.to(device), device)
print(model)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6417, 256)
    (lstm): LSTM(256, 512)
  )
  (decoder): Decoder(
    (embedding): Embedding(6417, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (lstm): LSTM(256, 512)
    (out): Linear(in_features=512, out_features=6417, bias=True)
  )
)


In [28]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        
        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta
        

    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')
                
        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')
                
            
        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

In [29]:
LR = 1e-3
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

es = EarlyStopping(patience=10, 
                   delta=0.001, 
                   mode='min', 
                   verbose=True)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min', 
                                                 factor=0.5, 
                                                 patience=2,
                                                 threshold_mode='abs',
                                                 min_lr=1e-8, 
                                                 verbose=True)

In [30]:
def train(model, dataloader, optimizer, loss_func, device):
    model.train()
    
    avg_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        output = model(x, y)
        output_dim = output.size(2)

        output = output.reshape(-1, output_dim)
        y = y.view(-1)

        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()

        avg_loss += loss.item() * x.size(0)

    return avg_loss / len(dataloader)


def evaluate(model, data_loader, loss_fn, device):
    model.eval()
    
    eval_loss = 0
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            output = model(x, y)
            output_dim = output.size(2)
            output = output.reshape(-1, output_dim)
            y = y.view(-1)
            
            # Loss 계산
            loss = loss_fn(output, y)
            
            eval_loss += loss.item() * x.size(0)
            
    return eval_loss / len(data_loader)

In [31]:
def sequence_to_sentence(sequences, index2word):
    outputs = []
    for p in sequences:

        word = index2word[p]
        if p not in [SOS_TOKEN, EOS_TOKEN, PAD_TOKEN]:
            outputs.append(word)
        if word == EOS_TOKEN:
            break
    return ' '.join(outputs)

def random_evaluation(model, dataset, index2word, device, n=10):
    
    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices
    
    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)
    
    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)        
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)
            
            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()
            
            for i in range(n):
                print(f'질문   : {sequence_to_sentence(x[i], index2word)}')
                print(f'답변   : {sequence_to_sentence(y[i], index2word)}')
                print(f'예측답변: {sequence_to_sentence(preds[i].argmax(1), index2word)}')
                print('==='*10)

In [32]:
NUM_EPOCHS = 100
STATEDICT_PATH = '/home/pervinco/Models/seq2seq-chatbot-kor.pt'

best_loss = np.inf
for epoch in range(NUM_EPOCHS):
    loss = train(model, train_dataloader, optimizer, loss_func, device)
    
    val_loss = evaluate(model, test_dataloader, loss_func, device)
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), STATEDICT_PATH)
    
    if epoch % 5 == 0:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')
    
    # Early Stop
    es(loss)
    if es.early_stop:
        break
    
    # Scheduler
    scheduler.step(val_loss)
                   
model.load_state_dict(torch.load(STATEDICT_PATH))
torch.save(model.state_dict(), f'/home/pervinco/Models/seq2seq-chatbot-kor-{best_loss:.4f}.pt')

epoch: 1, loss: 17.0216, val_loss: 0.9497
[EarlyStopping] (Update) Best Score: 17.02159
[EarlyStopping] (Update) Best Score: 14.36581
[EarlyStopping] (Update) Best Score: 13.91539
[EarlyStopping] (Update) Best Score: 13.47932
[EarlyStopping] (Update) Best Score: 13.18599
epoch: 6, loss: 12.8502, val_loss: 0.8486
[EarlyStopping] (Update) Best Score: 12.85022
[EarlyStopping] (Update) Best Score: 12.57749
[EarlyStopping] (Update) Best Score: 12.34305
[EarlyStopping] (Update) Best Score: 12.20058
[EarlyStopping] (Update) Best Score: 11.93401
epoch: 11, loss: 11.8160, val_loss: 0.8224
[EarlyStopping] (Update) Best Score: 11.81597
[EarlyStopping] (Update) Best Score: 11.56410
[EarlyStopping] (Update) Best Score: 11.30364
Epoch 00013: reducing learning rate of group 0 to 5.0000e-04.
[EarlyStopping] (Update) Best Score: 10.96245
[EarlyStopping] (Update) Best Score: 10.74508
epoch: 16, loss: 10.6937, val_loss: 0.8114
[EarlyStopping] (Update) Best Score: 10.69366
[EarlyStopping] (Update) Best Sc

In [33]:
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 현재 를 즐기 면서 살 고 싶 다 .
답변   : 현재 를 즐기 면서 행복 하 게 사세요 .
예측답변: 저 는 게 잘못 인 거 같 아요 .
질문   : 5 년 만났 어
답변   : 좋 은 기억 들 이 많 겠 네요 .
예측답변: 좋 은 하 는 있 어요 .
질문   : 상황 이 힘들 어서 헤어진다
답변   : 그럴 수 있 어요 .
예측답변: 저 도 요 .
질문   : 그 말 하 지 말 걸
답변   : 이불 킥 하 고 말 했 던 사실 조 차 잊어버리 세요 .
예측답변: 좋 은 하 이 .
질문   : 이 별후 2 주반 이 지난 시점
답변   : 수 많 은 것 들 이 달라졌 겠 죠 .
예측답변: 마음 의 준비 를 찾아보 세요 .
질문   : 여유 가 너무 없 어
답변   : 나 를 사랑 하 고 상대 를 사랑 하 는 여유 를 갖 게 되 길 바라 요 .
예측답변: 저 는 노래 를 찾아보 세요 .
질문   : 컨디션 조절 이 안 돼
답변   : 컨디션 조절 은 정말 중요 해요 .
예측답변: 마음 이 이 하 는 .
질문   : 우연히 또 만났 어
답변   : 우연 이 필연 이 되 기 도 합니다 .
예측답변: 저 도 는 해 보 세요 .
질문   : 바래다 달 라고 말 해 볼까 ?
답변   : 귀엽 게 부탁 해 보 세요 .
예측답변: 마음 하 는 게 .
질문   : 너 는 뭐 억 었 어 ?
답변   : 저 는 배터리 가 밥 이 예요 .
예측답변: 잘 하 고 있 어요 .
